In [2]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
num_time_shifts = 100
num_velocities = 32
min_time_shift = 0.01

In [18]:
data = np.load('gen_history.npy', allow_pickle=True)
instruments = np.load('gen_instruments.npy', allow_pickle=True)

In [19]:
recording = pretty_midi.PrettyMIDI()

# Note that channels are sorted by np.unique
channels, idx = np.unique(data[:, 1], return_index=True, axis=0)
for i in instruments:
    recording.instruments.append(pretty_midi.Instrument(program=i,name=pretty_midi.program_to_instrument_name(i)))
    
curr_time = 0
curr_velocity = [96 for channel in range(len(channels))]
#start time and velocity of the current note being played
note_start = [{} for channels in range(len(channels))]
note_velocity = [{} for channel in range(len(channels))]
note_same = [{} for channel in range(len(channels))]
for m, idx in enumerate(data[:, 0]):
    channel = data[m, 1]
    if idx < num_notes:
        #note on event
        if idx in note_start[channel]:
            note_same[channel][idx] += 1
            continue
        note_start[channel][idx] = curr_time
        note_velocity[channel][idx] = curr_velocity[channel]
        note_same[channel][idx] = 1
    elif idx < 2*num_notes:
        #note off event
        curr_note = idx - num_notes
        if curr_note not in note_start[channel]:
            continue
            #raise ValueError("note" + str(idx-num_notes) + "has not been started!")
        note_same[channel][curr_note] -= 1
        if note_same[channel][curr_note] == 0:
            note=pretty_midi.Note(velocity=note_velocity[channel][curr_note], pitch=curr_note, start=note_start[channel][curr_note], end=curr_time)
            recording.instruments[channel].notes.append(note)
            del note_start[channel][curr_note]
            del note_velocity[channel][curr_note]
            del note_same[channel][curr_note]
    elif idx < 2*num_notes + num_velocities:
        #velocity event
        #reversing the quantization in preprocess operation
        curr_velocity[channel] = int((idx-2*num_notes)*128/num_velocities)
    elif idx < 2*num_notes + num_time_shifts + num_velocities:
        #time shift event
        curr_time += (idx - 2*num_notes - num_velocities + 1)*min_time_shift

In [20]:
recording.write("gen_midi.mid")

In [16]:
file = open( "preprocessed_data_unified_fnames.p", "rb" )
res = pickle.load(file)
file.close()

In [17]:
print(res[100])

Beethoven/2416_lvbsext3.mid
